# Create Monthly Task Reports in Smartsheet


In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import json
import logging
from datetime import datetime, timedelta
from dateutil import parser
from typing import Dict, List
from pprint import pprint
from pathlib import Path
import jinja2
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import smartsheet
import scrapbook as sb
import dotenv

from jsonschema import validate, ValidationError, RefResolver
from jsonschema.exceptions import RefResolutionError

from docxtpl import (
    DocxTemplate,
    InlineImage,
)  # For this you'll need to `pip install docxtpl`

# This comes in with the templating library
from docx.shared import Inches
from prefect import task, flow

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

In [3]:
unposted_sheet_id = 4818113414883204
posted_sheet_id = 3567675495475076

<IPython.core.display.Javascript object>

In [4]:
environment = "dev"

# change these or set as papermill parameters to report on year and month
year_for_report = 2022
month_for_report = 5

<IPython.core.display.Javascript object>

In [5]:
# check the environment vars for secrets

env_file = f".env-{environment}"
logger.info("Loading the .env file from %s", env_file)
dotenv.load_dotenv(dotenv.find_dotenv(env_file))

assert os.environ.get("SMARTSHEET_KEY"), f"SMARTSHEET_KEY not found in {env_file}"

<IPython.core.display.Javascript object>

In [6]:
if not os.path.exists("_cache"):
    os.mkdir("_cache")

<IPython.core.display.Javascript object>

In [7]:
# connect smartsheet client
ss_client = smartsheet.Smartsheet(os.environ.get("SMARTSHEET_KEY"))
ss_client.errors_as_exceptions(True)

<IPython.core.display.Javascript object>

In [8]:
def validate_json(schema, json_data):

    # mostly from https://stackoverflow.com/questions/25145160/json-schema-ref-does-not-work-for-relative-path
    schemas = (json.load(open(source)) for source in Path("schema").iterdir())
    schema_store = {schema["$id"]: schema for schema in schemas}

    resolver = RefResolver.from_schema(schema, store=schema_store)

    try:
        result = validate(instance=json_data, schema=schema, resolver=resolver)
        return True, None
    except ValidationError as err:
        return False, err.message

<IPython.core.display.Javascript object>

In [9]:
def month_end_date(year, month):
    """Calculate the month end date given a year and month."""
    month += 1
    if month == 13:
        month = 1
        year += 1

    tempdate = datetime.strptime(f"{year}-{month}-1", "%Y-%m-%d")
    return (tempdate - timedelta(days=1)).strftime("%Y-%m-%d")

<IPython.core.display.Javascript object>

In [10]:
prepared_date = datetime.today().strftime("%Y-%m-%d")
prepared_date

'2022-07-11'

<IPython.core.display.Javascript object>

In [11]:
month_ending_date = month_end_date(year_for_report, month_for_report)
month_ending_date

'2022-05-31'

<IPython.core.display.Javascript object>

In [12]:
posted_time_sheet = ss_client.Sheets.get_sheet(posted_sheet_id)

<IPython.core.display.Javascript object>

In [13]:
# break down the cell IDs into a quick lookup box
posted_cell_ids = ["Row ID"]
for column in posted_time_sheet.columns:
    my_column = column.to_dict()
    posted_cell_ids.append(my_column["title"])

posted_cell_ids

['Row ID',
 'Month-end Date',
 'Project Title',
 'Account/Client',
 'Resource',
 'Completed Hours',
 'Notes',
 'Grant Proposal #',
 'Estimated Only',
 'MDC Account ID',
 'MDC Project ID',
 'MDC Resource ID',
 'Posted Date']

<IPython.core.display.Javascript object>

In [14]:
# break down the cells into a list of lists for a later dataframe
rows_list = []
for row in posted_time_sheet.rows:
    row_list = [row.id]
    for cell in row.cells:
        if cell.display_value:
            row_list.append(cell.display_value)
        else:
            # just in case there's a None in here, use NaN instead
            if cell.value:
                row_list.append(cell.value)
            else:
                row_list.append(np.NaN)

    rows_list.append(row_list)

<IPython.core.display.Javascript object>

In [15]:
# put it together as a dataframe
df = pd.DataFrame(rows_list, columns=posted_cell_ids)
df["Estimated Only"].fillna(False, inplace=True)
df

,Row ID,Month-end Date,Project Title,Account/Client,Resource,Completed Hours,Notes,Grant Proposal #,Estimated Only,MDC Account ID,MDC Project ID,MDC Resource ID,Posted Date
0,5341306060203908,2021-09-30,TISLab: Monarch UI (3.0) Redesign,HealthAI: TISLab,Vincent Rubinetti,56,Vince's 20% from Sept-end of March covered thi...,213359,False,1882681714,1882442059,25815860,2022-07-08
1,885083143333764,2021-10-31,TISLab: Monarch UI (3.0) Redesign,HealthAI: TISLab,Vincent Rubinetti,152,To Discuss with MH - 3/30/22 HH,213359,False,1882681714,1882442059,25815860,2022-07-08
2,2613624943863684,2021-11-30,Greenelab: lab-website-template and related si...,HealthAI: Greene Lab,Vincent Rubinetti,18,NaN,NaN,False,1882681138,1882712838,25815860,2022-07-08
3,7117224571234180,2021-11-30,Greenelab: Preprint Single Page App,HealthAI: Greene Lab,Vincent Rubinetti,5,NaN,NaN,False,1882681138,1977980999,25815860,2022-07-08
4,1487725037021060,2021-11-30,TISLab: Monarch UI (3.0) Redesign,HealthAI: TISLab,Vincent Rubinetti,4.30694,To Discuss with MH - 3/30/22 HH,213359,False,1882681714,1882442059,25815860,2022-07-08
5,2509459638904708,2021-12-31,Way: Grant Support,HealthAI: Way Lab,Faisal Alquaddoomi,2,NaN,NaN,False,1883649981,1957293587,25815853,2022-07-08
6,8842646614894468,2021-12-31,"Greenelab: Biomedical Literature ""Word Lapse"" ...",HealthAI: Greene Lab,Steve Taylor,2,NaN,213269,False,1882681138,1969468997,25810257,2022-07-08
7,4761259452589956,2021-12-31,Way: Grant Support,HealthAI: Way Lab,Steve Taylor,2,NaN,NaN,False,1883649981,1957293587,25810257,2022-07-08
8,257659825219460,2021-12-31,TISLab: Monarch UI (3.0) Redesign,HealthAI: TISLab,Vincent Rubinetti,48,To Discuss with MH - 3/30/22 HH,213359,False,1882681714,1882442059,25815860,2022-07-08
9,6321826692786052,2022-01-31,TISLab: Graph DB Deployer,HealthAI: TISLab,Faisal Alquaddoomi,19.31889,NaN,NaN,False,1882681714,1882913862,25815853,2022-07-08


<IPython.core.display.Javascript object>

In [16]:
# only need this months info and no estimates
monthly_df = (
    df.loc[
        ((df["Month-end Date"] == month_ending_date) & (df["Estimated Only"] == False))
    ]
    .copy()
    .fillna("")
)
monthly_df["Completed Hours"] = monthly_df["Completed Hours"].astype(float)
monthly_df.drop(columns=["Month-end Date", "Estimated Only"], inplace=True)
monthly_df

,Row ID,Project Title,Account/Client,Resource,Completed Hours,Notes,Grant Proposal #,MDC Account ID,MDC Project ID,MDC Resource ID,Posted Date
34,7922314580256644,Way Lab: Staffing/Support FY2022,HealthAI: Way Lab,Dave Bunten,16.0,,,1883649981,2334955423,29667633,2022-07-08
35,2292815046043524,"Greenelab: Biomedical Literature ""Word Lapse"" ...",HealthAI: Greene Lab,Faisal Alquaddoomi,11.0,,213269,1882681138,1969468997,25815853,2022-07-08
36,4544614859728772,Greenelab: mygeneset.info,HealthAI: Greene Lab,Vincent Rubinetti,48.0,,,1882681138,1882738595,25815860,2022-07-08
37,6796414673414020,"Greenelab: Biomedical Literature ""Word Lapse"" ...",HealthAI: Greene Lab,Vincent Rubinetti,39.0,,213269,1882681138,1969468997,25815860,2022-07-08
38,1166915139200900,Greenelab: Staffing/Support FY2022,HealthAI: Greene Lab,Vincent Rubinetti,18.0,,,1882681138,2303324267,25815860,2022-07-08
39,5670514766571396,TISLab: Monarch UI (3.0) Redesign,HealthAI: TISLab,Vincent Rubinetti,0.0,,213359,1882681714,1882442059,25815860,2022-07-08
40,3418714952886148,Way Lab: Staffing/Support FY2022,HealthAI: Way Lab,Vincent Rubinetti,8.0,,,1883649981,2334955423,25815860,2022-07-08


<IPython.core.display.Javascript object>

## With the posted hours dataframe, start breaking it down into a dict

In [17]:
accounts_df = (
    monthly_df.groupby(["MDC Account ID"])
    .agg({"Account/Client": "first", "Completed Hours": "sum"})
    .reset_index()
)
accounts_df

,MDC Account ID,Account/Client,Completed Hours
0,1882681138,HealthAI: Greene Lab,116.0
1,1882681714,HealthAI: TISLab,0.0
2,1883649981,HealthAI: Way Lab,24.0


<IPython.core.display.Javascript object>

In [18]:
projects_df = (
    monthly_df.groupby(["MDC Account ID", "MDC Project ID"])
    .agg(
        {
            "Project Title": "first",
            "Notes": "first",
            "Grant Proposal #": "first",
            "Completed Hours": "sum",
        },
    )
    .reset_index()
)
projects_df

,MDC Account ID,MDC Project ID,Project Title,Notes,Grant Proposal #,Completed Hours
0,1882681138,1882738595,Greenelab: mygeneset.info,,,48.0
1,1882681138,1969468997,"Greenelab: Biomedical Literature ""Word Lapse"" ...",,213269,50.0
2,1882681138,2303324267,Greenelab: Staffing/Support FY2022,,,18.0
3,1882681714,1882442059,TISLab: Monarch UI (3.0) Redesign,,213359,0.0
4,1883649981,2334955423,Way Lab: Staffing/Support FY2022,,,24.0


<IPython.core.display.Javascript object>

In [19]:
resources_df = (
    monthly_df.groupby(["MDC Account ID", "MDC Project ID", "MDC Resource ID"])
    .agg(
        {"Resource": "first", "Completed Hours": "sum", "Row ID": "first"},
    )
    .reset_index()
)

resources_df

,MDC Account ID,MDC Project ID,MDC Resource ID,Resource,Completed Hours,Row ID
0,1882681138,1882738595,25815860,Vincent Rubinetti,48.0,4544614859728772
1,1882681138,1969468997,25815853,Faisal Alquaddoomi,11.0,2292815046043524
2,1882681138,1969468997,25815860,Vincent Rubinetti,39.0,6796414673414020
3,1882681138,2303324267,25815860,Vincent Rubinetti,18.0,1166915139200900
4,1882681714,1882442059,25815860,Vincent Rubinetti,0.0,5670514766571396
5,1883649981,2334955423,25815860,Vincent Rubinetti,8.0,3418714952886148
6,1883649981,2334955423,29667633,Dave Bunten,16.0,7922314580256644


<IPython.core.display.Javascript object>

In [20]:
def get_task_file(project_id, resource_id, row_id):
    """
    Fetch the attachment in the form project_id-resource_id-month_ending_date.json
    from the given row_id.

    Returns a sorted by date and validated JSON/dict and completed hours per resource
    """

    with open(os.path.join("schema", "effort_hours-resource.json"), "r") as file:
        schema = json.load(file)

    # get the attachements
    response = ss_client.Attachments.list_row_attachments(
        posted_sheet_id, row_id, include_all=True
    )

    if response.data and len(response.data) > 0:
        for file in response.data:
            filename = f"{project_id}_{resource_id}_{month_ending_date}.json"
            if file.name == filename:
                # if the file name matches what we know the task list is,
                attachment = ss_client.Attachments.get_attachment(
                    posted_sheet_id, file.id
                )
                logger.info(f"Downloading {filename}...")
                ss_client.Attachments.download_attachment(attachment, "_cache")

                with open(os.path.join("_cache", filename), "r") as f:
                    json_result = json.load(f)

                # need to sum hours, and sort
                df = pd.DataFrame(json_result["Tasks"])
                completed_hours = df["Completed Hours"].sum()
                json_result["Tasks"] = df.sort_values(
                    by="Task Complete Date", ascending=True
                ).to_dict("records")

                # the files _must_ match the schema
                valid, msg = validate_json(schema, json_result)
                if not valid:
                    logger.error("validation error %s", msg)

                return json_result, completed_hours


# _ = get_task_file("1969468997", "25815853", "2292815046043524")

<IPython.core.display.Javascript object>

In [21]:
def get_client(client_id):

    client_dict = {}

    row = accounts_df.loc[accounts_df["MDC Account ID"] == client_id].to_dict("records")

    # should only be one
    if len(row) == 1:

        client_dict = {
            "MDC Client ID": row[0]["MDC Account ID"],
            "Client Name": row[0]["Account/Client"],
            "Notes": "",
            "No Bill": False,
            "Completed Hours": row[0]["Completed Hours"],
        }

    return client_dict


# get_client("1882681138")

<IPython.core.display.Javascript object>

In [22]:
def get_projects(client_id):

    with open(os.path.join("schema", "effort_hours-project.json"), "r") as file:
        schema = json.load(file)

    projects_dict = []

    rows = projects_df.loc[projects_df["MDC Account ID"] == client_id].to_dict(
        "records"
    )

    for row in rows:

        project_dict = {
            "MDC Project ID": row["MDC Project ID"],
            "Project Name": row["Project Title"],
            "Notes": row["Notes"],
            "Completed Hours": row["Completed Hours"],
            "Resources": [],
        }

        # get the resources by project
        resources_dict = resources_df.loc[
            (resources_df["MDC Project ID"] == row["MDC Project ID"])
        ].to_dict("records")

        # go through each resource
        for resource in resources_dict:

            resource_dict, completed_hours = get_task_file(
                row["MDC Project ID"], resource["MDC Resource ID"], resource["Row ID"]
            )

            resource_dict["Completed Hours"] = completed_hours
            project_dict["Resources"].append(resource_dict)

        valid, msg = validate_json(schema, project_dict)
        if not valid:
            logger.error("validation error %s", msg)

        projects_dict.append(project_dict)

    return projects_dict


# get_projects("1882681138")

<IPython.core.display.Javascript object>

## build a list of reports

In [23]:
def build_report(account_id):

    with open(os.path.join("schema", "effort_hours-report.json"), "r") as file:
        schema = json.load(file)

    logger.info(f"Creating report for account {account_id}.")

    # primary report dict
    report = {
        "Month-ending Date": month_ending_date,
        "Report Prepared Date": prepared_date,
        "Notes": "",
        "Client": {},
    }

    client_dict = get_client(account_id)

    projects = get_projects(client_dict["MDC Client ID"])
    # add the projects
    client_dict["Projects"] = projects

    # set the client in the report
    report["Client"] = client_dict

    valid, msg = validate_json(schema, report)
    if not valid:
        logger.error("The report is not valid, %s", msg)

    return report


# build_report("1882681138")

<IPython.core.display.Javascript object>

In [24]:
account_ids = accounts_df["MDC Account ID"].unique().tolist()
account_ids

['1882681138', '1882681714', '1883649981']

<IPython.core.display.Javascript object>

In [25]:
# build them all
account_reports = {}
for account_id in account_ids:
    report = build_report(account_id)
    account_reports[account_id] = report

<IPython.core.display.Javascript object>

In [30]:
# Create a docx template instance from the template Word file
template = DocxTemplate("client_report_template.docx")

account_keys = account_reports.keys()

for account_id in account_keys:

    # render the object in memory
    template.render({"v": account_reports[account_id]})

    # save the object to the file system
    filename = f"effort_hours-{account_id}-{month_ending_date}.docx"
    template.save(os.path.join("_cache", filename))

<IPython.core.display.Javascript object>